# Automated ML

In this notebook, we will use AutoML to train many different models and do hyperparameter tuning to give trading signals on Bitcoin price movements. To achieve this,

We will start by importing necessary dependencies and setting up our AzureML workspace details, along with our experiment and environment. Detailed package dependencies can be found on the [`env.yml`](envs/env.yml). Use `conda install --file envs/env.yml` on your Terminal. We end this first part by creating the compute cluster we will use to train and tune out model.

In [1]:
from azureml.core import Workspace, Experiment, Environment, Datastore, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

import pandas as pd
import joblib
import os
import requests

In [2]:
 # Setting up the workspace
ws = Workspace.from_config()

# Registering and building the environment (not needed in AutoML)
env = Environment.from_conda_specification(name = "azcapstone", file_path = "envs/env.yml")
env = env.register(workspace=ws)
env_build = env.build(workspace=ws)

# Setup the experiment
exp_name = 'az-capstone-automl'
exp=Experiment(ws, exp_name)

# Enable logs
run = exp.start_logging()

We now deploy the necessary Compute Cluster, or check if there is already an existing one we can use.

In [3]:
# Setup the compute cluster
compute_name = os.environ.get('CLUSTER_NAME', 'automl-cluster')
compute_min_nodes = os.environ.get('CLUSTER_MIN_NODES', 0)
compute_max_nodes = os.environ.get('CLUSTER_MAX_NODES', 4)
vm_size = os.environ.get('CLUSTER_SKU', 'STANDARD_D2_V2')

# Verify if the compute cluster exists
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(
        vm_size=vm_size,
        min_nodes=compute_min_nodes,
        max_nodes=compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    # poll for a minimum number of nodes and for a specific timeout.
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

creating a new compute target...
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2022-02-18T17:03:40.473000+00:00', 'errors': None, 'creationTime': '2022-02-18T17:03:40.072032+00:00', 'modifiedTime': '2022-02-18T17:03:43.640872+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Dataset

### Overview

The dataset we are using will be the one resulting from the [previous notebook](1-data-sourcing.ipynb) where
we dug into data sourcing and did some processing prior this task. The dataset
consists on financial data including OHLCV (open, high, low, close, volume) from diverse instruments (indices,
commodities, interest rates...) and technical indicators (moving averages, RSI, standard deviation...), that we will
use to create a ML-based trading model, that gives BUY, HOLD or SELL signals for Bitcoin trading.

If you want to dig more into how the dataset looks like or
into how the above-mentioned signals are generated, please refer to the "labelling the data" section of
the [data sourcing notebook](1-data-sourcing.ipynb) or the latest  print view of the DataFrame's head and/or tail
provided on the same file.

The task we will be trying to solve is basically a **classification problem**.
We are to predict whether the next-day, Bitcoin returns will be on the top 25% most positive returns (BUY, 1),
the 25% most negative (SELL, -1), or somewhere in between (HOLD, 0).

Since AutoML does grid search over features and normalization procedures, we will take joint, unaltered data as feed
in to the model. What we will make is dropping the last features and labels that are not really needed for the task.

In [4]:
# Access the data and drop unneeded columns for AutoML exercise
df = pd.read_csv('data/df.csv')
print('dataset shape: ', df.shape)
print('columns:\n', df.columns)

dataset shape:  (2708, 34)
columns:
 Index(['Date', 'shangai', 'btc', 'crude oil', 'euro', 'gold', 'silver', 'ftse',
       'spy', 'hsi', 'nasdaq', 'nikkei', 'rates', 'open', 'high', 'low', 'MA4',
       'MA50', 'MA80', 'stochRSI', 'RSI', 'btc_std_dev', 'std_dif', 'vol_btc',
       'hashrate', 'difficulty', 'transactions', 't_cost', 'y_returns',
       'y_close', 'y_c', 'y_returns_shift', 'y_c_shift', 'y_close_shift'],
      dtype='object')


In [5]:
drop_col_list = ['y_close', 'y_close_shift', 'y_returns_shift'] # our label will be y_c_shift
df.drop(columns=drop_col_list, inplace=True)

In [6]:
# Register the dataset
datastore = ws.get_default_datastore()
dataset = Dataset.Tabular.register_pandas_dataframe(df, datastore, "automl_dataset", show_progress=True)
df = dataset.to_pandas_dataframe()

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/8cd670b6-d3f4-4ab1-ba5a-8de10a9f04c3/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


## AutoML Configuration

Our AutoML run will have the classification task of predicting next day's buy, sell or hold label, or the column
`y_c_shift`. Our primary metric will be accuracy, as we need a metric we can compare later with HyperDrive
and considering that our problem doesn't have particular implications on false positives or negatives,
that would make us opt for recall or precision instead.
 I'm also adding the automatic featurization, so
AutoML takes care of necessary data transformations, trying out different methods.

As timeout for this project I will use 30 minutes. The usage of VMs to access Azure on a limited time (1h) adds pressure
on this metric. We also need time to analize results afterwards, so air time using the VM is important.

In [7]:
automl_settings = {"featurization": 'auto'}

In [8]:
# Set parameters for AutoMLConfig
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=df,
    label_column_name='y_c_shift',
    n_cross_validations=5,
    **automl_settings)

In [9]:
# Submit the experiment run
automl_run = exp.submit(automl_config, show_output=True)
automl_run.wait_for_completion(show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


2022-02-18:17:03:53,402 INFO     [modeling_bert.py:226] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2022-02-18:17:03:53,408 INFO     [modeling_xlnet.py:339] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2022-02-18:17:03:56,561 INFO     [utils.py:159] NumExpr defaulting to 4 threads.


Experiment,Id,Type,Status,Details Page,Docs Page
az-capstone-automl,AutoML_7948808a-fc20-4b91-881c-b5703f6f4c46,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the

2022-02-18:17:21:42,207 INFO     [explanation_client.py:334] Using default datastore for uploads


Experiment,Id,Type,Status,Details Page,Docs Page
az-capstone-automl,AutoML_7948808a-fc20-4b91-881c-b5703f6f4c46,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization
DETAILS:      
+------------------------------+------------------------------+------------------------------+
|Column name          

{'runId': 'AutoML_7948808a-fc20-4b91-881c-b5703f6f4c46',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2022-02-18T17:03:58.030749Z',
 'endTimeUtc': '2022-02-18T17:19:57.663165Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"az-capstone-automl","subscription_id":"510b94ba-e453-4417-988b-fbdc37b55ca7","resource_group":"aml-quickstarts-186961","workspace_name":"quick-starts-ws-186961","region":"southcentralus","compute_target":"local","spark_service":

## Run Details

By using the RunDetails widget, we can appreciate different experiments metrics.

OPTIONAL: Write about the different models trained and their performance.
Why do you think some models did better than others?

In [10]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

After training and tuning with AutoML, we are interested on the best performing model. We will get it as an output in the following lines of code, and display its properties.

In [11]:
# Retrieve and save best model
best_run, model = automl_run.get_output()
print(best_run, '\n')
print(model)
joblib.dump(value=best_run.id, filename="./inference/bitcoin-automl.joblib")

Run(Experiment: az-capstone-automl,
Id: AutoML_7948808a-fc20-4b91-881c-b5703f6f4c46_30,
Type: None,
Status: Completed) 

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=None))], verbose=False))], flatten_transform=None, weights=[0.07142857142857142, 0.21428571428571427, 0.07142857142857142, 0.14285714285714285, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142]))],
         verbose=False)
Y_transformer(['LabelEncoder', LabelEncoder()])


['./inference/bitcoin-automl.joblib']

## Model Deployment

We will deploy the AutoML model as an REST endpoint. For this, we will first register the model, and then download a `score.py` script used for scoring, as well as a `conda_env.yml` file that will be used in our compute instance to install dependencies. We will also download the model as a pickle file. All these files can be found in the [`inference`](inference/) directory.

Before deploying the model, we will pass in required scripts, dependencies and configuration for a compute instance, as an inference config object, and then deploy the model as a web service.

In [12]:
model_name = 'best-automl-model'
description = "AutoML model for predicting day-ahead Bitcoin price movements"
tags = None
model = automl_run.register_model(model_name=model_name, description=description, tags=tags)
print(automl_run.model_id)

best-automl-model


In [13]:
from azureml.core.model import Model

In [14]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
az-capstone-automl,AutoML_7948808a-fc20-4b91-881c-b5703f6f4c46_30,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [15]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'inference/conda_env.yml')
best_run.download_file("outputs/scoring_file_v_1_0_0.py", "inference/score.py")
best_run.download_file('outputs/model.pkl', 'inference/model.pkl')

In [24]:
print('Best Run Id: ', best_run.id)
best_run_metrics = best_run.get_metrics()

for i in best_run_metrics:
    print(i, best_run_metrics[i])

Best Run Id:  AutoML_7948808a-fc20-4b91-881c-b5703f6f4c46_30
f1_score_macro 0.3932451452725806
weighted_accuracy 0.640275976978873
recall_score_macro 0.41115600837480615
matthews_correlation 0.17132098348584746
precision_score_micro 0.5262217705356351
precision_score_macro 0.45813101026236486
recall_score_weighted 0.5262217705356351
average_precision_score_weighted 0.5183918800041305
balanced_accuracy 0.41115600837480615
precision_score_weighted 0.4859983537055637
f1_score_weighted 0.46626047840808776
AUC_micro 0.6997457470652353
recall_score_micro 0.5262217705356351
f1_score_micro 0.5262217705356351
average_precision_score_micro 0.5643399854001072
average_precision_score_macro 0.4574498688241723
accuracy 0.5262217705356351
AUC_macro 0.6494884239833302
AUC_weighted 0.6620469687706404
log_loss 0.9863956063713729
norm_macro_recall 0.11673401256220928
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_7948808a-fc20-4b91-881c-b5703f6f4c46_30/confusion_matrix
accuracy_table aml://a

In [16]:
script_file_name = "./inference/score.py"
env = Environment.from_conda_specification(name="automl-env", file_path="inference/conda_env.yml")

inference_config = InferenceConfig(environment=env, entry_script=script_file_name)

aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=2,
    memory_gb=2,
    tags={"area": "Trading", "type": "automl_classification"},
    description="service for Bitcoin trading signals"
)

aci_service_name = model_name.lower()
print(aci_service_name)

aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

aci_service.get_logs()

best-automl-model
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-02-18 17:28:38+00:00 Creating Container Registry if not exists.
2022-02-18 17:28:39+00:00 Registering the environment.
2022-02-18 17:28:40+00:00 Use the existing image.
2022-02-18 17:28:40+00:00 Generating deployment configuration.
2022-02-18 17:28:40+00:00 Submitting deployment to compute.
2022-02-18 17:28:43+00:00 Checking the status of deployment best-automl-model..
2022-02-18 17:31:25+00:00 Checking the status of inference endpoint best-automl-model.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


'2022-02-18T17:31:16,083583000+00:00 - rsyslog/run \n2022-02-18T17:31:16,083632700+00:00 - gunicorn/run \nDynamic Python package installation is disabled.\nStarting HTTP server\n2022-02-18T17:31:16,085402800+00:00 - iot-server/run \n2022-02-18T17:31:16,086866600+00:00 - nginx/run \nrsyslogd: /azureml-envs/azureml_4943656157e05766fedf432d8be562a4/lib/libuuid.so.1: no version information available (required by rsyslogd)\nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2022-02-18T17:31:16,309166600+00:00 - iot-server/finish 1 0\n2022-02-18T17:31:16,311364000+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 20.1.0\nListening at: http://127.0.0.1:31311 (84)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 112\nSPARK_HOME not set. Skipping PySpark Initialization.\nGenerating new fontManager, this may take some time...\nInitializing logger\n2022-02-18 17:31:18,873 | root | INFO | Starting up app insights client

Ok, so everything seems to be working fine. 

Now we will just test the endpoint serving our model with some test data. We need to consider that the score.py file will expect the data in a certain format.
AzureML expects the data columns to be in alphabetical order. In the lines below, we make necesary transformations and then 
pass in our data as JSON and ping our endpoint with Python's request library. If everything goes well, we get a response from the API and some
predictions within it in form of a list.

In [20]:
import json

print(f'Will test endpoint: {aci_service.scoring_uri}')
X_test_json = df[sorted(df.columns)].drop(columns=['y_c_shift']).tail().to_json(orient="records")

data = '{"data": ' + X_test_json + "}"
headers = {"Content-Type": "application/json"}

Will test endpoint: http://518b0eb2-b0fe-472b-9e58-0d276d32d3d3.southcentralus.azurecontainer.io/score


In [21]:
resp = requests.post(aci_service.scoring_uri, data, headers=headers)
y_pred = json.loads(json.loads(resp.text))
print(y_pred)

{'result': [0.0, 0.0, 0.0, 0.0, 0.0]}


### Deletion of endpoints and resources

In this notebook we've created both a compute cluster and a compute instance that needs to be taken down. In the following cell we delete the used resources as a closure.

In [22]:
# Deleting the inference compute instance
aci_service.delete()
print('Compute cluster deleted!')

# Deleting compute cluster
compute_target.delete()
print('Compute cluster deleted!')

Compute cluster deleted!
Compute cluster deleted!


**Submission Checklist**
- (DONE) I have registered the model.
- (DONE) I have deployed the model with the best accuracy as a webservice.
- (DONE) I have tested the webservice by sending a request to the model endpoint.
- (DONE) I have deleted the webservice and shutdown all the computes that I have used.
- (DONE) I have taken a screenshot showing the model endpoint as active.
- (DONE) The project includes a file containing the environment details (see the inference directory).
